In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minzzzzi","key":"74a980fb52033cf0ead89c84120a4a9c"}'}

In [2]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle

Uninstalling kaggle-1.5.8:
  Successfully uninstalled kaggle-1.5.8
     |████████████████████████████████| 1.5MB 2.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.9-py3-none-any.whl size=73265 sha256=19175705785a577f2c933a93247c4716cac33b7566743dd0e346de4abeaf22a7
  Stored in directory: /root/.cache/pip/wheels/9d/50/3d/2644504bb1e8c782f3fef5984f03d76fc4a74698fdec128b29
Successfully built kaggle


In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c rainyseason

  0% 0.00/9.12k [00:00<?, ?B/s]
100% 9.12k/9.12k [00:00<00:00, 3.63MB/s]


In [8]:
!unzip rainyseason.zip

Archive:  rainyseason.zip
  inflating: rainyseason-sample.csv  
  inflating: rainyseason-test.csv    
  inflating: rainyseason-train.csv   


In [34]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter

In [12]:
train = pd.read_csv('rainyseason-train.csv',header=None,skiprows=1)
test = pd.read_csv('rainyseason-test.csv',header=None,skiprows=1)
submit = pd.read_csv('rainyseason-sample.csv')

train[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1973,강릉,1973-06-25,1973-06-30,1,1,1,0,1,0,0,0,1
1,1977,강릉,1977-06-23,1977-07-19,1,1,1,0,1,1,1,0,1
2,1979,강릉,1979-06-19,1979-07-23,1,1,1,0,1,1,1,0,0
3,1981,강릉,1981-06-17,1981-07-14,1,1,1,0,1,1,1,0,1
4,1983,강릉,1983-06-19,1983-07-25,0,1,1,0,1,1,1,0,1


In [29]:
x_train = train.loc[:,5:12]
y_train = train.loc[:,4]

x_test = test.loc[:,5:12]

In [35]:
print(Counter(y_train))

Counter({1: 643, 0: 426})


In [36]:
sm = SMOTE(random_state=1)
x_train_syn, y_train_syn = sm.fit_resample(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


---
#KNN
---

In [38]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 5,p=2)

KNN.fit(x_train_syn,y_train_syn)
train_pred = KNN.predict(x_train_syn)
print(accuracy_score(y_train_syn,train_pred))

0.5637636080870918


In [39]:
test_pred = KNN.predict(x_test)

for i in range(len(test_pred)):
  submit['RainySeason'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index = None)
!kaggle competitions submit -c rainyseason -f submission.csv -m "KNN"

100% 180/180 [00:05<00:00, 34.8B/s]
Successfully submitted to 2020.Spring.AI.Termproject_장마 예측하기

#### (n_neighbors = 5,p=2) /0.65714
#### (n_neighbors = 10,p=2) / 0.57
#### (n_neighbors = 5,p=2) + syn / 0.71428

---
#Logistic Regression
---

In [44]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(random_state = 1,C = 1.0)

logit.fit(x_train_syn,y_train_syn)

test_pred = logit.predict(x_test)

for i in range(len(test_pred)):
  submit['RainySeason'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index = None)
!kaggle competitions submit -c rainyseason -f submission.csv -m "logit"

100% 180/180 [00:07<00:00, 23.2B/s]
Successfully submitted to 2020.Spring.AI.Termproject_장마 예측하기

#### 0.1 + syn + balance / 0.54
#### 1.0 + syn / 0.54

---
# LDA QDA
---

In [46]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()

qda.fit(x_train_syn,y_train_syn)

test_pred = qda.predict(x_test)

for i in range(len(test_pred)):
  submit['RainySeason'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index = None)
!kaggle competitions submit -c rainyseason -f submission.csv -m "qda"

100% 180/180 [00:03<00:00, 48.1B/s]
Successfully submitted to 2020.Spring.AI.Termproject_장마 예측하기

---
# Decision Tree
---

In [50]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(random_state=1,criterion='gini')

dtc.fit(x_train,y_train)

test_pred = dtc.predict(x_test)

for i in range(len(test_pred)):
  submit['RainySeason'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index = None)
!kaggle competitions submit -c rainyseason -f submission.csv -m "dtc"

100% 180/180 [00:05<00:00, 31.0B/s]
Successfully submitted to 2020.Spring.AI.Termproject_장마 예측하기